In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
data = np.genfromtxt('raw_data.txt', delimiter = ',')
X = data[:,:4]
y = data[:, 4]
print(X)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.T
y_train = y_train.reshape(1, y_train.shape[0])
X_test = X_test.T
y_test = y_test.reshape(1, y_test.shape[0])
print ('Train X Shape: ', X_train.shape)
print ('Train Y Shape: ', y_train.shape)
print ('I have m = %d training examples!' % (X_train.shape[1]))

print ('\nTest X Shape: ', X_test.shape)

In [ ]:
def define_structure(X, Y):
    input_unit = X.shape[0] # size of input layer
    hidden_unit = 1 #hidden layer of size 4
    output_unit = Y.shape[0] # size of output layer
    return (input_unit, hidden_unit, output_unit)
(input_unit, hidden_unit, output_unit) = define_structure(X_train, y_train)
print("The size of the input layer is:  = " + str(input_unit))
print("The size of the hidden layer is:  = " + str(hidden_unit))
print("The size of the output layer is:  = " + str(output_unit))

In [ ]:
def parameters_initialization(input_unit, hidden_unit, output_unit):
    np.random.seed(2) 
    weight1 = np.random.randn(hidden_unit, input_unit)*0.01
    bias1 = np.zeros((hidden_unit, 1))
    weight2 = np.random.randn(output_unit, hidden_unit)*0.01
    bias2 = np.zeros((output_unit, 1))
    parameters = {"W1": weight1,
                  "b1": bias1,
                  "W2": weight2,
                  "b2": bias2}
    
    return parameters

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
def forward_propagation(X, parameters):
    weight1 = parameters['W1']
    bias1 = parameters['b1']
    weight2 = parameters['W2']
    bias2 = parameters['b2']
    netInput1 = np.dot(weight1, X) + bias1
    output1 = np.tanh(netInput1)
    netInput2 = np.dot(weight2, output1) + bias2
    output2 = sigmoid(netInput2)
    cache = {"Z1": netInput1,"Y1": output1,"Z2": netInput2,"Y2": output2}
    
    return output2, cache

In [ ]:
def delta(calculatedOutput, targetOutput, parameters):
    # number of training example
    m = targetOutput.shape[1] 
    # Compute the cross-entropy cost
    logprobs = np.multiply(np.log(calculatedOutput), targetOutput) + np.multiply((1-targetOutput), np.log(1 - calculatedOutput))
    error = - np.sum(logprobs) / m
    error = float(np.squeeze(error))
                                    
    return error

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    #number of training example
    m = X.shape[1]
    
    weight1 = parameters['W1']
    weight2 = parameters['W2']
    output1 = cache['Y1']
    output2 = cache['Y2']
   
    deltaZ2 = output2-Y
    deltaW2 = (1/m) * np.dot(deltaZ2, output1.T)
    deltab2 = (1/m) * np.sum(deltaZ2, axis=1, keepdims=True)
    deltaZ1 = np.multiply(np.dot(weight2.T, deltaZ2), 1 - np.power(output1, 2))
    deltaW1 = (1/m) * np.dot(deltaZ1, X.T) 
    deltab1 = (1/m)*np.sum(deltaZ1, axis=1, keepdims=True)
    
    grads = {"dW1": deltaW1, "db1": deltab1, "dW2": deltaW2,"db2": deltab2}
    
    return grads

In [ ]:
def updated_parameters(parameters, grads, learning_rate = 0.01):
    weight1 = parameters['W1']
    bias1 = parameters['b1']
    weight2 = parameters['W2']
    bias2 = parameters['b2']
   
    deltaW1 = grads['dW1']
    deltab1 = grads['db1']
    deltaW2 = grads['dW2']
    deltab2 = grads['db2']

    weight1 = weight1 - learning_rate * deltaW1
    bias1 = bias1 - learning_rate * deltab1
    weight2 = weight2 - learning_rate * deltaW2
    bias2 = bias2 - learning_rate * deltab2
    
    parameters = {"W1": weight1, "b1": bias1,"W2": weight2,"b2": bias2}
    
    return parameters

In [ ]:
def neural_network_model(X, Y, hidden_unit, num_iterations = 10000):
    np.random.seed(3)
    input_unit = define_structure(X, Y)[0]
    output_unit = define_structure(X, Y)[2]
    
    parameters = parameters_initialization(input_unit, hidden_unit, output_unit)
   
    weight1 = parameters['W1']
    bias1 = parameters['b1']
    weight2 = parameters['W2']
    bias2 = parameters['b2']
    
    for i in range(0, num_iterations):
        output, cache = forward_propagation(X, parameters)
        error = delta(output, Y, parameters)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = updated_parameters(parameters, grads)
        if i % 100 == 0:
            print ("Loss after iteration %i: %f" %(i, error))
    return parameters
parameters = neural_network_model(X_train, y_train, 1, num_iterations=10000)

In [ ]:
def prediction(parameters, X):
    output, cache = forward_propagation(X, parameters)
    predictions = np.round(output)
    
    return predictions

In [ ]:
predictions = prediction(parameters, X_train)
print ('Accuracy Train: %d' % float((np.dot(y_train, predictions.T) + np.dot(1 - y_train, 1 - predictions.T))/float(y_train.size)*100) + '%')
predictions = prediction(parameters, X_test)
print ('Accuracy Test: %d' % float((np.dot(y_test, predictions.T) + np.dot(1 - y_test, 1 - predictions.T))/float(y_test.size)*100) + '%')

In [ ]:
new_values1 = [3.6216, 8.6661, -2.8073, -0.44699]
new_values2 = [0.066129,2.4914,-2.9401,-0.62156]

new_prediction1 = prediction(parameters, new_values1)
new_prediction2 = prediction(parameters, new_values2)
print(new_prediction1)
print(new_prediction2)